In [10]:
#Import data of stocks in S&P 500 present from 1999 to 2019
import yfinance as yf
import numpy as np
import pandas as pd
import nasdaqdatalink
import requests
from bs4 import BeautifulSoup

NASDAQ_DATA_LINK_API_KEY = "TqyaGpQ_6SKSx38euQUU"
# Get the S&P 500 constituents
sp500_constituents_url = "https://en.wikipedia.org/wiki/List_of_S&P_500_companies"
sp500_constituents_df = pd.read_html(sp500_constituents_url)[0]
#print(sp500_constituents_df["Symbol"])
symbols = sp500_constituents_df["Symbol"]


In [11]:
# Filter out the stocks in ticker list that are not present in S&P 500 from 1999 to 2019
# Get the data of the stocks in S&P 500 from 1999 to 2019
print(f'Original Length: {len(symbols)}')
#symbols = ['AAPL']
tickers = []
for symbol in symbols:
    try: 
        data = yf.download(symbol, start="1999-01-01", end="2019-12-31", progress=False)
        #print(data)
        #data.index = data['Date']
        #print(data.head())
        #print(data.index[0].year)
        #print(data.index[0].month)
        #print(data.index[0].day)
        #print(data.index[-1].year)
        #print(data.index[-1].month)
        #print(data.index[-1].day)
        
        if [data.index[0].year, data.index[0].month, data.index[0].day] == [1999, 1, 4] and [data.index[-1].year, data.index[-1].month, data.index[-1].day] == [2019, 12, 30]:
            #print("Yes")
            tickers.append(symbol)
            
            # Add Returns column
            data['Returns'] = data['Adj Close'].pct_change() + 1
            data.dropna(inplace=True)
            data.to_csv(f"data/{symbol}.csv")
        
    except:
        continue
        
print(f'Length after filtering: {len(tickers)}')  
    

Original Length: 502



1 Failed download:
['ABNB']: Exception("%ticker%: Data doesn't exist for startDate = 915166800, endDate = 1577768400")

1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1999-01-01 -> 2019-12-31)')

1 Failed download:
['CARR']: Exception("%ticker%: Data doesn't exist for startDate = 915166800, endDate = 1577768400")

1 Failed download:
['CEG']: Exception("%ticker%: Data doesn't exist for startDate = 915166800, endDate = 1577768400")

1 Failed download:
['GEHC']: Exception("%ticker%: Data doesn't exist for startDate = 915166800, endDate = 1577768400")

1 Failed download:
['KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 915166800, endDate = 1577768400")

1 Failed download:
['OGN']: Exception("%ticker%: Data doesn't exist for startDate = 915166800, endDate = 1577768400")

1 Failed download:
['OTIS']: Exception("%ticker%: Data doe

Length after filtering: 348


In [12]:
# Download S&P 500 data from 1999 to 2019 as baseline data
data = yf.download("^GSPC", start="1999-01-01", end="2019-12-31", interval="1d")
data['Market Returns'] = data['Adj Close'].pct_change()
data.dropna(inplace=True)
data.to_csv(f"data/GSPC.csv")

[*********************100%%**********************]  1 of 1 completed


In [14]:

import os
import pandas as pd
import statsmodels.api as sm

DATA_DIR = (os.getcwd() + "/data/")
import requests 
from bs4 import BeautifulSoup
start_date = '1999-01-01'
end_date = "2019-12-31"
# Get fundamental data for each stock in the ticker and append to the dataframe

def get_all_symbols():
    return [v.strip('.csv') for v in os.listdir(DATA_DIR)]

tickers = get_all_symbols()
#tickers = ['AAPL']
market_data = pd.read_csv(f"data/GSPC.csv", index_col = 0)


for ticker in tickers:
    ticker_data = pd.read_csv(f"data/{ticker}.csv", index_col=0)
    print(f"Current Ticker: {ticker}")
    #Skip if columns exists
    if ticker == 'GSPC' or 'Short Term Reversal' in ticker_data.columns.tolist():
        continue
    #Short Term Reversal, Stock Momentum, Long Term Reversal
    st_period = 5
    lt_period = 20
    ticker_data['Short Term Reversal'] = ticker_data['Close'].pct_change(
        periods=st_period)
    
    ticker_data['Stock Momentum'] = (
        ticker_data['Close'] / ticker_data['Close'].shift(st_period)) - 1
    
    ticker_data['Long Term Reversal'] = ticker_data['Close'].pct_change(periods=lt_period)
    

    # Calculate Market Beta for each stock    
    market_betas = []
    for date in ticker_data.index:
        # Filter market data for the specific date
        market_data_for_date = market_data[market_data.index <= date]

        # Calculate Market Beta for that date
        X = sm.add_constant(market_data_for_date['Market Returns'])
        y = ticker_data.loc[ticker_data.index <= date, 'Returns']
        model = sm.OLS(y, X).fit()
        beta = model.params['Market Returns']
        market_betas.append(beta)
        
    # Add Market Beta as a new column in the stock data
    ticker_data['Market_Beta'] = market_betas
    
    
    # Calculating Turnover, Turnover Volatility, and Total Return Volatility
    
    # Taking Volume as Turnover
    ticker_data['Turnover Volatility'] = ticker_data['Volume'].rolling(window=lt_period).std()
    
    # Add Dividends
    ticker_historical = yf.Ticker(ticker)
    historical = ticker_historical.history(period='1mo', start=start_date, end=end_date)
    
    ticker_dividends = ticker_historical.dividends
    ticker_data['Dividends'] = ticker_dividends
    ticker_data['Dividends'].fillna(0.0, inplace=True)
    
    ticker_data['Total Returns'] = ticker_data['Close'].pct_change() + (ticker_data['Dividends'] / ticker_data['Close'])
    ticker_data['Total Return Volatility'] = ticker_data['Total Returns'].rolling(window=lt_period).std()
        
    ticker_data.dropna(inplace=True)
    ticker_data.to_csv(f"data/{ticker}.csv")
    #print(ticker_data.head())
        
        # Get the fundamental data
        #data_url = f"https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}"
        #data_response = requests.get(data_url)
        #data = data_response.content.decode()
        #soup = BeautifulSoup(data, 'html.parser')
        #print(soup.prettify())
        #print(soup.find_all("div", {"class": "D(tbr) fi-row Bgc($hoverBgColor):h"}))
    #print(ticker_data)
    #current_year, current_month = ticker
    #data_url = f"https://finance.yahoo.com/quote/{symbol}?p={symbol}"
    #data_response = requests.get(data_url)

    #data = data_response.content.decode()
    #market_cap_start_index = data.find("Market Cap:")
    #market_cap_end_index = data.find(
    #    "</span>", market_cap_start_index)
    #market_cap = data[market_cap_start_index +
    #                             12:market_cap_end_index].strip()

    


Current Ticker: AAPL
Current Ticker: ABT
Current Ticker: ACGL
Current Ticker: ADBE
Current Ticker: ADI
Current Ticker: ADM
Current Ticker: ADP
Current Ticker: ADSK
Current Ticker: AEE
Current Ticker: AEP
Current Ticker: AES
Current Ticker: AFL
Current Ticker: AIG
Current Ticker: AJG
Current Ticker: ALB
Current Ticker: ALK
Current Ticker: ALL
Current Ticker: AMAT
Current Ticker: AMD
Current Ticker: AME
Current Ticker: AMGN
Current Ticker: AMT
Current Ticker: AMZN
Current Ticker: ANSS
Current Ticker: AON
Current Ticker: AOS
Current Ticker: APA
Current Ticker: APD
Current Ticker: APH
Current Ticker: ARE
Current Ticker: ATO
Current Ticker: AVB
Current Ticker: AVY
Current Ticker: AXP
Current Ticker: AZO
Current Ticker: BA
Current Ticker: BAC
Current Ticker: BALL
Current Ticker: BAX
Current Ticker: BBWI
Current Ticker: BBY
Current Ticker: BDX
Current Ticker: BEN
Current Ticker: BIIB
Current Ticker: BIO
Current Ticker: BK
Current Ticker: BKR
Current Ticker: BMY
Current Ticker: BRO
Current Tic